In [ ]:
!pip install insightface opencv-python-headless onnxruntime

In [ ]:
import insightface
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import cv2
import os
import glob
import torch

# Load the InsightFace model
model = insightface.app.FaceAnalysis(name='buffalo_l')  # Pretrained model
model.prepare(ctx_id=0, det_size=(112, 112))

def get_embedding(image_path):
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        return None

    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return None

    # Preprocess image: resize, normalize, transpose
    img = cv2.resize(img, (112, 112))  # Resize to model input size
    img = img[..., ::-1]  # Convert BGR to RGB
    img = np.transpose(img, (2, 0, 1))  # HWC -> CHW
    img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
    img = np.expand_dims(img, axis=0).astype(np.float32)  # Add batch dimension

    # Compute embedding using ONNX model
    with torch.no_grad():
        embedding = model.models['recognition'].forward(img)[0]

    # We are returning feature vector for the whole image, because we already have a cropped face. However, you could use FaceAnalysis' face detection too.
    return embedding



# Build known_faces gallery
faces_dir = "faces"
known_faces = {}
for label in ['a', 'b', 'c']:
    files = glob.glob(os.path.join(faces_dir, f"{label}*.jpg"))
    embeddings = [get_embedding(f) for f in files if get_embedding(f) is not None]
    known_faces[label] = np.mean(embeddings, axis=0) if embeddings else None

# Recognition function
def recognize_face(test_image_path):
    test_emb = get_embedding(test_image_path)
    if test_emb is None: return "Err"
    scores = {name: cosine_similarity([test_emb], [ref_emb])[0][0] for name, ref_emb in known_faces.items() if ref_emb is not None}
    best_match = max(scores, key=scores.get)
    best_score = scores[best_match]
    return best_match, best_score


# Make predictions
test_images = ["test-face-a.jpg", "test-face-b.jpg"]
for img in test_images:
    identity, score = recognize_face(img)
    print(f"Predicted identity for {img}: {identity} (Score: {score:.4f})")
